In [1]:
# study N_orb & N_r relation
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.ischeckResolved import ischeckResolved3d
from utils.plot3dtree import plot3dtree

tol_list = numpy.array([1.0e-02, 1.0e-03, 1.0e-4])
p_list = numpy.array([4,5,6])
basis_list = ['cc-pVDZ', # Double-zeta
              'cc-pVTZ', # Triple-zeta
              'cc-pVQZ'] # Quadruple-zeta, https://en.wikipedia.org/wiki/Basis_set_(chemistry)
Nr_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Err_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Level_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
npts = 10
xx0, yy0, zz0 = numpy.meshgrid(numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),indexing='ij') # double check later...

print("H2: ")
for i in range(tol_list.size):
  tol = tol_list[i]
  print("===========================================================")
  print("tol is: ", tol)
  for j in range(p_list.size):
    p = p_list[j]
    print("----------------------------------------------------------- ")
    print("order is: ", p)
    for k in range(len(basis_list)):
      # basis
      mol_h2 = gto.M(
          verbose = 0,
          atom = '''
          h    0    0.       0.
          h    0    0.       0.74''',
          basis = basis_list[k]) 
      dom = numpy.array([-11, 11, -11, 11, -11, 11])
      checkpts = numpy.array([[ 0,    0.,       0.],
                              [ 0,    0.,    0.74]])
      ifcoeffs = False
      ifstorecoeffs = False
      def pyscffunc(x, y, z, mol):
        valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        nrows, ncols = valstmp.shape
        vals = numpy.zeros((nrows, ncols**2))
        for j in range(ncols):
          for k in range(ncols):
              vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
        return vals
      # # tree based on proxy func
      # def pyscffunc(x, y, z, mol):
      #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
      #   nrows, ncols = valstmp.shape
      #   vals = numpy.zeros((nrows))
      #   for j in range(ncols):
      #     vals = vals + valstmp[:, j]**2
      #   return vals
      func = lambda x, y, z: pyscffunc(x, y, z, mol_h2)
      tree_h2 = {
          'domain': dom.reshape(-1,1), 
          'tol': tol,
          'nSteps': 15,
          'n': p,
          'checkpts': checkpts.transpose(),
          'ifcoeffs': ifcoeffs,
          'ifstorecoeffs': ifstorecoeffs                     
        }
      tree_h2, rint = buildBreadthFirst3d(tree_h2, func)
      Nr_list[i][j][k] = p**3*numpy.sum(tree_h2['height']==0)
      # error for checkfunc
      def pyscffunc2(x, y, z, mol):
        valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        nrows, ncols = valstmp.shape
        vals = numpy.zeros((nrows, ncols))
        offidx = 1
        for j in range(ncols):
          k = numpy.remainder(j+offidx, ncols)
          vals[:, j] = valstmp[:, j] * valstmp[:, k]
        return vals
      func2 = lambda x, y, z: pyscffunc2(x, y, z, mol_h2)
      testval = func2(xx0.flatten(),yy0.flatten(),zz0.flatten())
      _,nd = testval.shape
      tree_h2, checkerror = ischeckResolved3d(tree_h2, func2, nd)
      Err_list[i][j][k] = numpy.max(checkerror)
      Level_list[i][j][k] = numpy.max(tree_h2['level'])
      print("-------------------------- ")
      print("basis is: ", basis_list[k])
      print("Nr is: ", Nr_list[i][j][k])
      print("testfunc error is: ", Err_list[i][j][k])
      print("level is: ", Level_list[i][j][k])
      

# plot3dtree(tree_h2)

H2: 
tol is:  0.01
----------------------------------------------------------- 
order is:  4
-------------------------- 
basis is:  cc-pVDZ
Nr is:  25600.0
testfunc error is:  0.0028789097746713004
level is:  6.0
-------------------------- 
basis is:  cc-pVTZ
Nr is:  43520.0
testfunc error is:  0.0049413762565583335
level is:  7.0
-------------------------- 
basis is:  cc-pVQZ
Nr is:  65024.0
testfunc error is:  0.004792900340898737
level is:  7.0
----------------------------------------------------------- 
order is:  5
-------------------------- 
basis is:  cc-pVDZ
Nr is:  36000.0
testfunc error is:  0.0007920435412899248
level is:  5.0
-------------------------- 
basis is:  cc-pVTZ
Nr is:  50000.0
testfunc error is:  0.004144981496617062
level is:  6.0
-------------------------- 
basis is:  cc-pVQZ
Nr is:  67500.0
testfunc error is:  0.009646213262835705
level is:  7.0
----------------------------------------------------------- 
order is:  6
-------------------------- 
basis is:  cc-

In [ ]:
# p**3*numpy.sum(tree_h2['height']==0)
# Nr_list[0,:,:].transpose(),Nr_list[1,:,:].transpose(),Nr_list[2,:,:].transpose()


In [ ]:
# study N_orb & N_r relation
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.ischeckResolved import ischeckResolved3d
from utils.plot3dtree import plot3dtree

tol_list = numpy.array([1.0e-02, 1.0e-03, 1.0e-4])
p_list = numpy.array([4,5,6])
basis_list = ['cc-pVDZ', # Double-zeta
              'cc-pVTZ', # Triple-zeta
              'cc-pVQZ'] # Quadruple-zeta, https://en.wikipedia.org/wiki/Basis_set_(chemistry)
Nr_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Err_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Level_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
npts = 10
xx0, yy0, zz0 = numpy.meshgrid(numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),indexing='ij') # double check later...

print("HF: ")
for i in range(tol_list.size):
  tol = tol_list[i]
  print("===========================================================")
  print("tol is: ", tol)
  for j in range(p_list.size):
    p = p_list[j]
    print("----------------------------------------------------------- ")
    print("order is: ", p)
    for k in range(len(basis_list)):
      if not (p == 4 and tol == 1.0e-04 and basis_list[k] == 'cc-pVQZ'): # for some reason, this one takes too long...
        # basis
        mol_hf = gto.M(
            verbose = 0,
            atom = '''
            h    0    0.       0.
            f    0    0.       0.92''',
            basis = basis_list[k]) 
        dom = numpy.array([-13, 13, -13, 13, -13, 13])
        checkpts = numpy.array([[ 0,    0.,       0.],
                                [ 0,    0.,    0.92]])
        ifcoeffs = False
        ifstorecoeffs = False
        # tree
        def pyscffunc(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols**2))
          for j in range(ncols):
            for k in range(ncols):
                vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
          return vals
        # # tree based on proxy func
        # def pyscffunc(x, y, z, mol):
        #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        #   nrows, ncols = valstmp.shape
        #   vals = numpy.zeros((nrows))
        #   for j in range(ncols):
        #     vals = vals + valstmp[:, j]**2
        #   return vals
        func = lambda x, y, z: pyscffunc(x, y, z, mol_hf)
        tree_hf = {
            'domain': dom.reshape(-1,1), 
            'tol': tol,
            'nSteps': 15,
            'n': p,
            'checkpts': checkpts.transpose(),
            'ifcoeffs': ifcoeffs,
            'ifstorecoeffs': ifstorecoeffs                     
          }
        tree_hf, rint = buildBreadthFirst3d(tree_hf, func)
        Nr_list[i][j][k] = p**3*numpy.sum(tree_hf['height']==0)
        # error for checkfunc
        def pyscffunc2(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols))
          offidx = 1
          for j in range(ncols):
            k = numpy.remainder(j+offidx, ncols)
            vals[:, j] = valstmp[:, j] * valstmp[:, k]
          return vals
        func2 = lambda x, y, z: pyscffunc2(x, y, z, mol_hf)
        testval = func2(xx0.flatten(),yy0.flatten(),zz0.flatten())
        _,nd = testval.shape
        tree_hf, checkerror = ischeckResolved3d(tree_hf, func2, nd)
        Err_list[i][j][k] = numpy.max(checkerror)
        Level_list[i][j][k] = numpy.max(tree_hf['level'])
        print("-------------------------- ")
        print("basis is: ", basis_list[k])
        print("Nr is: ", Nr_list[i][j][k])
        print("testfunc error is: ", Err_list[i][j][k])
        print("level is: ", Level_list[i][j][k])
      

# plot3dtree(tree_hf)

In [ ]:
# p**3*numpy.sum(tree_hf['height']==0)
# Nr_list[0,:,:].transpose(),Nr_list[1,:,:].transpose(),Nr_list[2,:,:].transpose()

In [ ]:
# study N_orb & N_r relation
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.ischeckResolved import ischeckResolved3d
from utils.plot3dtree import plot3dtree

tol_list = numpy.array([1.0e-02, 1.0e-03, 1.0e-4])
p_list = numpy.array([4,5,6])
basis_list = ['cc-pVDZ', # Double-zeta
              'cc-pVTZ', # Triple-zeta
              'cc-pVQZ'] # Quadruple-zeta, https://en.wikipedia.org/wiki/Basis_set_(chemistry)
Nr_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Err_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Level_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
npts = 10
xx0, yy0, zz0 = numpy.meshgrid(numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),indexing='ij') # double check later...

print("H2O: ")
for i in range(tol_list.size):
  tol = tol_list[i]
  print("===========================================================")
  print("tol is: ", tol)
  for j in range(p_list.size):
    p = p_list[j]
    print("----------------------------------------------------------- ")
    print("order is: ", p)
    for k in range(len(basis_list)):
      if not (tol == 1.0e-04 and basis_list[k] == 'cc-pVQZ'): # for some reason, this one takes too long...
        # basis
        mol_h2o = gto.M(
            verbose = 0,
            atom = '''
            o    0    0.       0.
            h    0    -0.757   0.587
            h    0    0.757    0.587''',
            basis = basis_list[k]) 
        dom = numpy.array([-15, 15, -15, 15, -15, 15])
        checkpts = numpy.array([[ 0,    0.,       0.],
                                [ 0,    -0.757,   0.587],
                                [ 0,    0.757,    0.587]])
        ifcoeffs = False
        ifstorecoeffs = False
        # tree
        def pyscffunc(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols**2))
          for j in range(ncols):
            for k in range(ncols):
                vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
          return vals
        # # tree based on proxy func
        # def pyscffunc(x, y, z, mol):
        #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        #   nrows, ncols = valstmp.shape
        #   vals = numpy.zeros((nrows))
        #   for j in range(ncols):
        #     vals = vals + valstmp[:, j]**2
        #   return vals
        func = lambda x, y, z: pyscffunc(x, y, z, mol_h2o)
        tree_h2o = {
            'domain': dom.reshape(-1,1), 
            'tol': tol,
            'nSteps': 15,
            'n': p,
            'checkpts': checkpts.transpose(),
            'ifcoeffs': ifcoeffs,
            'ifstorecoeffs': ifstorecoeffs                     
          }
        tree_h2o, rint = buildBreadthFirst3d(tree_h2o, func)
        Nr_list[i][j][k] = p**3*numpy.sum(tree_h2o['height']==0)
        # error for checkfunc
        def pyscffunc2(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols))
          offidx = 1
          for j in range(ncols):
            k = numpy.remainder(j+offidx, ncols)
            vals[:, j] = valstmp[:, j] * valstmp[:, k]
          return vals
        func2 = lambda x, y, z: pyscffunc2(x, y, z, mol_h2o)
        testval = func2(xx0.flatten(),yy0.flatten(),zz0.flatten())
        _,nd = testval.shape
        tree_h2o, checkerror = ischeckResolved3d(tree_h2o, func2, nd)
        Err_list[i][j][k] = numpy.max(checkerror)
        Level_list[i][j][k] = numpy.max(tree_h2o['level'])
        print("-------------------------- ")
        print("basis is: ", basis_list[k])
        print("Nr is: ", Nr_list[i][j][k])
        print("testfunc error is: ", Err_list[i][j][k])
        print("level is: ", Level_list[i][j][k])


# plot3dtree(tree_h2o)

In [ ]:
# p**3*numpy.sum(tree_h2o['height']==0)
# Nr_list[0,:,:].transpose(),Nr_list[1,:,:].transpose(),Nr_list[2,:,:].transpose()

In [ ]:
# study N_orb & N_r relation
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.ischeckResolved import ischeckResolved3d
from utils.plot3dtree import plot3dtree

tol_list = numpy.array([1.0e-02, 1.0e-03, 1.0e-4])
p_list = numpy.array([4,5,6])
basis_list = ['cc-pVDZ', # Double-zeta
              'cc-pVTZ', # Triple-zeta
              'cc-pVQZ'] # Quadruple-zeta, https://en.wikipedia.org/wiki/Basis_set_(chemistry)
Nr_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Err_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Level_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
npts = 10
xx0, yy0, zz0 = numpy.meshgrid(numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),indexing='ij') # double check later...

print("CH4: ")
for i in range(tol_list.size):
  tol = tol_list[i]
  print("===========================================================")
  print("tol is: ", tol)
  for j in range(p_list.size):
    p = p_list[j]
    print("----------------------------------------------------------- ")
    print("order is: ", p)
    for k in range(len(basis_list)):
      if not (tol == 1.0e-04 and basis_list[k] == 'cc-pVQZ') and \
         not (tol == 1.0e-03 and basis_list[k] == 'cc-pVQZ'): # for some reason, this one takes too long...
        # basis
        mol_ch4 = gto.Mole()
        mol_ch4.atom = [ # CH4
            ['H',    0.0000,    0.9870,    0.0000],
            ['C',    1.0390,    0.9870,    0.0000],
            ['H',    1.0390,    1.9740,    0.0000],
            ['H',    1.0390,    0.0000,    0.0000],
            ['H',    2.0260,    0.9870,    0.0000]]
        mol_ch4.basis = {'H': basis_list[k],
                        'C': basis_list[k],}
        mol_ch4.build()
        dom = numpy.array([-11, 11, -11, 11, -11, 11])
        checkpts = numpy.array([[ 0,    0.,       0.],
                                [ 0,    0.,    0.74]])
        ifcoeffs = False
        ifstorecoeffs = False
        # tree
        def pyscffunc(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols**2))
          for j in range(ncols):
            for k in range(ncols):
                vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
          return vals
        # # tree based on proxy func
        # def pyscffunc(x, y, z, mol):
        #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        #   nrows, ncols = valstmp.shape
        #   vals = numpy.zeros((nrows))
        #   for j in range(ncols):
        #     vals = vals + valstmp[:, j]**2
        #   return vals
        func = lambda x, y, z: pyscffunc(x, y, z, mol_ch4)
        tree_ch4 = {
            'domain': dom.reshape(-1,1), 
            'tol': tol,
            'nSteps': 15,
            'n': p,
            'checkpts': checkpts.transpose(),
            'ifcoeffs': ifcoeffs,
            'ifstorecoeffs': ifstorecoeffs                     
          }
        tree_ch4, rint = buildBreadthFirst3d(tree_ch4, func)
        Nr_list[i][j][k] = p**3*numpy.sum(tree_ch4['height']==0)
        # error for checkfunc
        def pyscffunc2(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols))
          offidx = 1
          for j in range(ncols):
            k = numpy.remainder(j+offidx, ncols)
            vals[:, j] = valstmp[:, j] * valstmp[:, k]
          return vals
        func2 = lambda x, y, z: pyscffunc2(x, y, z, mol_ch4)
        testval = func2(xx0.flatten(),yy0.flatten(),zz0.flatten())
        _,nd = testval.shape
        tree_ch4, checkerror = ischeckResolved3d(tree_ch4, func2, nd)
        Err_list[i][j][k] = numpy.max(checkerror)
        Level_list[i][j][k] = numpy.max(tree_ch4['level'])
        print("-------------------------- ")
        print("basis is: ", basis_list[k])
        print("Nr is: ", Nr_list[i][j][k])
        print("testfunc error is: ", Err_list[i][j][k])
        print("level is: ", Level_list[i][j][k])

# plot3dtree(tree_ch4)

In [ ]:
# p**3*numpy.sum(tree_ch4['height']==0)
# Nr_list[0,:,:].transpose(),Nr_list[1,:,:].transpose(),Nr_list[2,:,:].transpose()

In [ ]:
# study N_orb & N_r relation
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.ischeckResolved import ischeckResolved3d
from utils.plot3dtree import plot3dtree

tol_list = numpy.array([1.0e-02, 1.0e-03, 1.0e-4])
p_list = numpy.array([4,5,6])
basis_list = ['cc-pVDZ', # Double-zeta
              'cc-pVTZ', # Triple-zeta
              'cc-pVQZ'] # Quadruple-zeta, https://en.wikipedia.org/wiki/Basis_set_(chemistry)
Nr_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Err_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
Level_list = numpy.zeros((tol_list.size,p_list.size,len(basis_list)))
npts = 10
xx0, yy0, zz0 = numpy.meshgrid(numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),numpy.linspace(0,1,npts),indexing='ij') # double check later...

print("C2H4: ")
for i in range(tol_list.size):
  tol = tol_list[i]
  print("===========================================================")
  print("tol is: ", tol)
  for j in range(p_list.size):
    p = p_list[j]
    print("----------------------------------------------------------- ")
    print("order is: ", p)
    for k in range(len(basis_list)):
      if not (basis_list[k] == 'cc-pVQZ') : # for some reason, this one takes too long...
        # basis
        mol_c2h4 = gto.M(
            verbose = 0,
            atom = '''
              C 0.000 0.000 0.000
              C 1.333 0.000 0.000
              H -0.176 1.109 0.000
              H -0.176 -1.109 0.000
              H 1.509 1.109 0.000
              H 1.509 -1.109 0.000''',
            basis = basis_list[k])
        dom = numpy.array([-17, 17, -17, 17, -17, 17])
        checkpts = numpy.array([[ 0.000, 0.000, 0.000],
                                [ 1.333, 0.000, 0.000],
                                [ -0.176, 1.109, 0.000],
                                [ -0.176, -1.109, 0.000],
                                [ 1.509, 1.109, 0.000],
                                [ 1.509, -1.109, 0.000]])
        ifcoeffs = False
        ifstorecoeffs = False
        # tree
        def pyscffunc(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols**2))
          for j in range(ncols):
            for k in range(ncols):
                vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
          return vals
        # # tree based on proxy func
        # def pyscffunc(x, y, z, mol):
        #   valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
        #   nrows, ncols = valstmp.shape
        #   vals = numpy.zeros((nrows))
        #   for j in range(ncols):
        #     vals = vals + valstmp[:, j]**2
        #   return vals
        func = lambda x, y, z: pyscffunc(x, y, z, mol_c2h4)
        tree_c2h4 = {
            'domain': dom.reshape(-1,1), 
            'tol': tol,
            'nSteps': 15,
            'n': p,
            'checkpts': checkpts.transpose(),
            'ifcoeffs': ifcoeffs,
            'ifstorecoeffs': ifstorecoeffs                     
          }
        tree_c2h4, rint = buildBreadthFirst3d(tree_c2h4, func)
        Nr_list[i][j][k] = p**3*numpy.sum(tree_c2h4['height']==0)
        # error for checkfunc
        def pyscffunc2(x, y, z, mol):
          valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
          nrows, ncols = valstmp.shape
          vals = numpy.zeros((nrows, ncols))
          offidx = 1
          for j in range(ncols):
            k = numpy.remainder(j+offidx, ncols)
            vals[:, j] = valstmp[:, j] * valstmp[:, k]
          return vals
        func2 = lambda x, y, z: pyscffunc2(x, y, z, mol_c2h4)
        testval = func2(xx0.flatten(),yy0.flatten(),zz0.flatten())
        _,nd = testval.shape
        tree_c2h4, checkerror = ischeckResolved3d(tree_c2h4, func2, nd)
        Err_list[i][j][k] = numpy.max(checkerror)
        Level_list[i][j][k] = numpy.max(tree_c2h4['level'])
        print("-------------------------- ")
        print("basis is: ", basis_list[k])
        print("Nr is: ", Nr_list[i][j][k])
        print("testfunc error is: ", Err_list[i][j][k])
        print("level is: ", Level_list[i][j][k])

# plot3dtree(tree_c2h4)

In [ ]:
# p**3*numpy.sum(tree_c2h4['height']==0)
# Nr_list[0,:,:].transpose(),Nr_list[1,:,:].transpose(),Nr_list[2,:,:].transpose()